In [161]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

/Users/ariankalantari/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [181]:
URL = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 40
page_size = 100


# Defining the target pages
target_page = f"{URL}/page/1/?sortby=post_date%3ADesc&pagesize=100"

# print(target_page)

page = requests.get(target_page)
soup = BeautifulSoup(page.content, "html.parser")

# Collect all reviews on the page
reviews_on_page = soup.find_all('article', itemprop="review")


def get_text_or_none(tag, selector, sibling=False):
    found = tag.find("td", class_=selector)
    if not found:
        return None
    return found.next_sibling.get_text(strip=True) if sibling else found.get_text(strip=True)

def get_rating(review, class_name):
    td = review.find("td", class_=class_name)
    if td:
        tr = td.find_parent("tr")
        stars = tr.find_all("span", class_="star fill")
        if stars:
            return stars[-1].get_text(strip=True)
    return None

In [187]:
posted_dates = []
verified_list = []
comments = []
aircrafts = []
traveller_types = []
cabin_classes = []
routes = []
flight_dates = []
seat_comfort_ratings = []
cabin_staff_service_ratings = []
food_and_beverages_ratings = []
inflight_entertainment_ratings = []
ground_service_ratings = []
wifi_and_connectivity_ratings = []
value_for_money_ratings = []
is_recommended_list = []




for i in range(1,pages+1):
    target_page = f"{URL}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # print(target_page)
    source = requests.get(target_page)
    soup = BeautifulSoup(source.content, "html.parser")

    # Collect all reviews on the page
    reviews_on_page = soup.find_all('article', itemprop="review")
    
    for review in reviews_on_page:
        # Posted Date
        date_posted = review.find('meta').get('content') if review.find('meta') else None

        # Verified
        verified_tag = review.find('div', itemprop="reviewBody")
        verified = verified_tag.find('em').get_text(strip=True) if verified_tag and verified_tag.find('em') else None

        # Comment
        comment = verified_tag.get_text(strip=True) if verified_tag else None

        # General Attributes
        aircraft = get_text_or_none(review, "aircraft", sibling=True)
        traveller_type = get_text_or_none(review, "type_of_traveller", sibling=True)
        cabin_class = get_text_or_none(review, "cabin_flown", sibling=True)
        route = get_text_or_none(review, "route", sibling=True)
        flight_date = get_text_or_none(review, "date_flown", sibling=True)

        # Ratings
        seat_comfort_rate = get_rating(review, "seat_comfort")
        cabin_staff_rate = get_rating(review, "cabin_staff_service")
        food_rate = get_rating(review, "food_and_beverages")
        entertainment_rate = get_rating(review, "inflight_entertainment")
        ground_service_rate = get_rating(review, "ground_service")
        wifi_rate = get_rating(review, "wifi_and_connectivity")
        value_rate = get_rating(review, "value_for_money")

        # Recommended
        recommended = get_text_or_none(review, "recommended", sibling=True)

        # Append all to lists
        posted_dates.append(date_posted)
        verified_list.append(verified)
        comments.append(comment)
        aircrafts.append(aircraft)
        traveller_types.append(traveller_type)
        cabin_classes.append(cabin_class)
        routes.append(route)
        flight_dates.append(flight_date)
        seat_comfort_ratings.append(seat_comfort_rate)
        cabin_staff_service_ratings.append(cabin_staff_rate)
        food_and_beverages_ratings.append(food_rate)
        inflight_entertainment_ratings.append(entertainment_rate)
        ground_service_ratings.append(ground_service_rate)
        wifi_and_connectivity_ratings.append(wifi_rate)
        value_for_money_ratings.append(value_rate)
        is_recommended_list.append(recommended)

In [188]:
review_data = pd.DataFrame({
    'posted_date': posted_dates,
    'is_verified': verified_list,
    'comment': comments,
    'aircraft': aircrafts,
    'traveller_type': traveller_types,
    'cabin_class': cabin_classes,
    'route': routes,
    'flight_date': flight_dates,
    'seat_comfort_rating': seat_comfort_ratings,
    'cabin_staff_service_rating': cabin_staff_service_ratings,
    'food_and_beverages_rating': food_and_beverages_ratings,
    'inflight_entertainment_rating': inflight_entertainment_ratings,
    'ground_service_rating': ground_service_ratings,
    'wifi_and_connectivity_rating': wifi_and_connectivity_ratings,
    'value_for_money_rating': value_for_money_ratings,
    'is_recommended': is_recommended_list
})

In [189]:
review_data

,posted_date,is_verified,comment,aircraft,traveller_type,cabin_class,route,flight_date,seat_comfort_rating,cabin_staff_service_rating,food_and_beverages_rating,inflight_entertainment_rating,ground_service_rating,wifi_and_connectivity_rating,value_for_money_rating,is_recommended
0,2025-03-31,Not Verified,Not Verified| Very good service on this route...,A320,Couple Leisure,Business Class,London Gatwick to Fuerteventura,March 2025,4,5,5,None,4,None,4,yes
1,2025-03-19,Trip Verified,✅Trip Verified| Flight mainly let down by a ...,Boeing 787,Solo Leisure,Business Class,Cape Town to London,March 2025,5,2,5,5,4,1,4,yes
2,2025-03-16,Trip Verified,✅Trip Verified| Another awful experience by ...,None,Business,Business Class,London to Geneva,December 2024,1,1,None,None,1,None,1,no
3,2025-03-16,Trip Verified,"✅Trip Verified| The service was rude, full o...",None,Business,Premium Economy,London to New York,January 2025,1,1,1,None,5,None,1,no
4,2025-03-16,Trip Verified,✅Trip Verified| This flight was a joke. Ther...,None,Business,Business Class,Chambery to London,January 2025,1,1,2,None,1,None,1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3920,2012-08-29,None,YYZ to LHR - July 2012 - I flew overnight in p...,None,None,Premium Economy,None,None,4,3,3,4,None,None,4,yes
3921,2012-08-28,None,LHR to HAM. Purser addresses all club passenge...,None,None,Business Class,None,None,4,5,4,None,None,None,3,yes
3922,2011-10-12,None,My son who had worked for British Airways urge...,None,None,Economy Class,None,None,None,None,None,None,None,None,4,yes
3923,2011-10-11,None,London City-New York JFK via Shannon on A318 b...,None,None,Premium Economy,None,None,1,3,5,None,None,None,1,no


In [190]:
len(review_data)

3925

In [193]:
review_data.to_csv('data.csv', index=False) 